In [1]:
import pandas as pd

#### BANCO VACINADOS #### 
df_vacinados = pd.read_csv('Arquivos/SESRS_VAC_APLICACAO_08_02_2022-08_54.csv', sep=';', encoding='cp1252')

/var/folders/4y/plk7ct8n4g9gz40s3_dvz7cm0000gn/T/ipykernel_24214/2976256361.py:4: DtypeWarning: Columns (2,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vacinados = pd.read_csv('Arquivos/SESRS_VAC_APLICACAO_08_02_2022-08_54.csv', sep=';', encoding='cp1252')


In [2]:
# Vacinados por município de residência
df_vac_mun = df_vacinados[['TP_VACINA', 'CD_MUNICIPIO', 'COUNT']]
df_vac_mun = df_vac_mun.groupby(by=['CD_MUNICIPIO']).sum().reset_index()
df_vac_mun['CD_MUNICIPIO'] = df_vac_mun['CD_MUNICIPIO'].astype('str')

In [3]:
# Arquivo com o nome dos municípios
df_nome_mun = pd.read_csv('Arquivos/SESRS - Imunização Covid-19RS (1).csv', sep=',', encoding='utf8', dtype='str')
df_nome_mun = df_nome_mun[['CRS', 'Município']]
df_nome_mun[['codigo', 'municipio']] = df_nome_mun['Município'].str.split(' - ', expand=True)
df_nome_mun = df_nome_mun[['CRS', 'codigo', 'municipio']]
df_nome_mun['codigo'] = df_nome_mun['codigo'].astype('str')

In [4]:
# Merge para pegar o nome pelo código IBGE
df_vac_mun = pd.merge(df_vac_mun, df_nome_mun, left_on='CD_MUNICIPIO', right_on='codigo', how='outer')
df_vac_mun = df_vac_mun[['municipio', 'COUNT']]

In [6]:
df_eventos = pd.read_excel('Relatorios/evento_adverso.xlsx')
data_eapv = df_eventos

In [7]:
# TOTAL EAPV POR MUNICIPIO
df_eapv_mun = data_eapv[['MUNICIPIO DE RESIDENCIA']]
df_eapv_mun['eapv'] = 1
df_eapv_mun = df_eapv_mun.groupby(by=['MUNICIPIO DE RESIDENCIA']).sum()
df_eapv_mun.reset_index(inplace=True)
#df_eapv_mun.to_excel('df_eapv_mun.xlsx')

/var/folders/4y/plk7ct8n4g9gz40s3_dvz7cm0000gn/T/ipykernel_24214/4284739177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eapv_mun['eapv'] = 1


In [8]:
# Acentuar os municípios para plotar no gráfico
# Ler o arquivo acentuado
df_eapv_mun_com_acentos = pd.read_excel('Arquivos/df_eapv_mun_com_acentos.xlsx') #'df_eapv_mun_com_acentos.xlsx'
df_eapv_mun_com_acentos.drop(columns=['Unnamed: 0'], inplace=True)
df_eapv_mun_com_acentos.rename(columns={'MUNICIPIO DE RESIDENCIA' : 'municipio'}, inplace=True)

In [9]:
# Merge EAPV com aplicadas por município
df_eapv_apl_mun = pd.merge(df_vac_mun, df_eapv_mun_com_acentos, on='municipio', how='outer')
df_eapv_apl_mun = df_eapv_apl_mun.fillna(0)

In [10]:
# Calculo da incidência por município
df_eapv_apl_mun['incidencia_1000ds'] = round((df_eapv_apl_mun['eapv'] / df_eapv_apl_mun['COUNT'] * 1000),2)
df_eapv_apl_mun

,municipio,COUNT,eapv,incidencia_1000ds
0,Aceguá,8943,4.0,0.45
1,Água Santa,6785,0.0,0.00
2,Agudo,30143,3.0,0.10
3,Ajuricaba,12554,5.0,0.40
4,Alecrim,12445,4.0,0.32
...,...,...,...,...
492,Vista Alegre do Prata,3405,0.0,0.00
493,Vista Gaúcha,5935,0.0,0.00
494,Vitória das Missões,6677,4.0,0.60
495,Westfalia,6143,1.0,0.16


In [11]:
##### Arquivo para MAPA ####
df_eapv_apl_mun_incid = df_eapv_apl_mun[['municipio', 'incidencia_1000ds']]
df_eapv_apl_mun_incid.to_excel('Arquivos/municipios_incidencia_EAPV.xlsx', index=False)

In [12]:
teste = df_eapv_apl_mun_incid.sort_values(by='incidencia_1000ds')
teste.tail(20)

,municipio,incidencia_1000ds
317,Pinto Bandeira,3.05
259,Mormaço,3.09
463,Tucunduva,3.30
274,Nova Bréscia,3.31
282,Nova Ramada,3.32
119,Cotiporã,3.79
302,Passa Sete,3.95
212,Itapuca,4.00
307,Pedras Altas,4.68
465,Tupanci do Sul,4.71
